In [5]:
import sys
sys.path.append('..')
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

# Figur erstellen
fig1 = go.FigureWidget()  # 2D Parallel Coordinates
fig2 = go.FigureWidget()  # 3D Visualisierung

# Dimensionssteuerung
dim_slider = widgets.IntSlider(
    value=3,
    min=2,
    max=6,
    step=1,
    description='Dimensionen:'
)

# Vektor-Eingabe (dynamisch basierend auf Dimensionen)
vector_sliders = []
for i in range(6):  # Maximal 6 Dimensionen
    slider = widgets.FloatSlider(
        value=1,
        min=-10,
        max=10,
        step=0.1,
        description=f'D{i+1}:',
        layout=widgets.Layout(width='400px')  # Breite festlegen
    )
    slider.layout.visibility = 'hidden'
    vector_sliders.append(slider)

def update_plots(*args):
    dims = dim_slider.value
    vector = [slider.value for slider in vector_sliders[:dims]]
    
    # Update 2D Parallel Coordinates
    with fig1.batch_update():
        fig1.data = []
        dimensions = [{
            'range': [-10, 10],
            'label': f'D{i+1}',
            'values': [vector[i]]
        } for i in range(dims)]
        
        fig1.add_trace(go.Parcoords(
            line=dict(color='blue'),
            dimensions=dimensions
        ))
        
        fig1.update_layout(
            title=f'{dims}D Vektor Visualisierung',
            showlegend=False,
            height=400
        )
    
    # Update 3D Visualization (wenn mindestens 3 Dimensionen)
    with fig2.batch_update():
        fig2.data = []
        if dims >= 3:
            fig2.add_scatter3d(
                x=[0, vector[0]],
                y=[0, vector[1]],
                z=[0, vector[2]],
                mode='lines+markers',
                line=dict(color='blue', width=4),
                name='3D Vektor'
            )
            
            # Koordinatenachsen
            for i, (color, name) in enumerate([('red', 'X'), ('green', 'Y'), ('blue', 'Z')]):
                fig2.add_scatter3d(
                    x=[-10, 10] if i==0 else [0,0],
                    y=[0,0] if i==0 else [-10, 10] if i==1 else [0,0],
                    z=[0,0] if i<2 else [-10, 10],
                    mode='lines',
                    line=dict(color=color, width=2),
                    name=f'{name}-Achse'
                )
            
            fig2.update_layout(
                scene=dict(
                    xaxis=dict(range=[-10, 10], showbackground=False),
                    yaxis=dict(range=[-10, 10], showbackground=False),
                    zaxis=dict(range=[-10, 10], showbackground=False),
                    aspectmode='cube'
                ),
                height=400
            )

# Event Handler
dim_slider.observe(update_plots, names='value')
for slider in vector_sliders:
    slider.observe(update_plots, names='value')

# Layout
slider_box = widgets.VBox([
    widgets.HTML(value='<h3>Dimensionen:</h3>'),
    dim_slider,
    widgets.HTML(value='<h3>Vektorkomponenten:</h3>'),
    widgets.VBox(vector_sliders)
])

container = widgets.VBox([
    slider_box,
    fig1,
    fig2
])

display(container)
update_plots()